## This code is the consolidation of just the relevant steps to train the model using Random Forest Algorithm and to save the corresponding pickle file locally.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import pickle

In [30]:
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [22]:
ai_data=pd.read_csv('Downloads/XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv')

In [23]:
ai_data_df=pd.DataFrame(ai_data)

In [45]:
def Data_preparation(df):
    
    # Add Meal_Type column ={'Breakfast', 'Lunch', 'Dinner'}
    time_to_meal_type={}
    for i in list(df['Time'].unique()):
        if i in (8,9,10,11):
            time_to_meal_type[i]='Breakfast'
        elif i in (12,13,14,15):
            time_to_meal_type[i]='Lunch'
        else:
            time_to_meal_type[i]='Evening_Snack'
    Meal_type=[]
    for i in list(ai_data_df['Time']):
        Meal_type.append(time_to_meal_type[i])
    df['Meal_type']=Meal_type
    
    #One Hot encode all categorical variables except Order column
    
    
    one_hot_encoded_data = pd.get_dummies(ai_data_df, columns = ['Year', 'University','Major','Time', 'Meal_type'])
    one_hot_encoded_data = one_hot_encoded_data.drop(columns=['Order'])
    
    return one_hot_encoded_data

In [46]:
Data_preparation(ai_data_df)

,Year_Year 1,Year_Year 2,Year_Year 3,Year_Year 4,University_Ball State University,University_Butler University,University_DePauw University,University_Indiana State University,University_Indiana University Bloomington,University_Indiana University-Purdue University Indianapolis (IUPUI),...,Time_11,Time_12,Time_13,Time_14,Time_15,Time_16,Time_17,Meal_type_Breakfast,Meal_type_Evening_Snack,Meal_type_Lunch
0,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4996,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
4997,0,0,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4998,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time=   1.4s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=   0.9s
[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   0.0s
[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   0.0s
[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=400; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth

In [47]:
X = one_hot_encoded_data.drop('Order', axis=1)
Y = one_hot_encoded_data['Order']
col_names= X.columns

In [48]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, stratify=Y)

In [50]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [51]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   1.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   1.3s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time=   1.8s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   3.0s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   3.8s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1400; total time=   3.1s


/Users/dhanyan/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
92 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/dhanyan/a

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [52]:
y_pred=rf_random.predict(X_test)

In [53]:
#The model takes in these optimised parameters to produce the final output
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [54]:
accuracy_score(Y_test, y_pred)

0.64

In [55]:
print(classification_report(Y_test, y_pred))


                                                          precision    recall  f1-score   support

                        Breaded Pork Tenderloin Sandwich       0.58      0.41      0.48        74
                                  Cornbread Hush Puppies       0.62      0.62      0.62        77
                                    Fried Catfish Basket       0.67      0.58      0.62        74
                        Hoosier BBQ Pulled Pork Sandwich       0.65      0.83      0.73        72
                 Indiana Buffalo Chicken Tacos (3 tacos)       0.58      0.50      0.54        74
    Indiana Corn on the Cob (brushed with garlic butter)       0.75      0.72      0.73        74
                                      Indiana Pork Chili       0.73      0.86      0.79        77
                                         Sugar Cream Pie       0.56      0.53      0.55        77
                                      Sweet Potato Fries       0.58      0.61      0.59        76
Ultimate Grilled Ch

In [56]:
#Create Pickle File
pickle.dump(model_rf,open('Downloads/Random_forest_model_Xtern_Dhanyan', 'wb'))

In [57]:
#Load the created pickle 
rf_model_loaded=pickle.load(open('Downloads/Random_forest_model_Xtern_Dhanyan', 'rb'))

In [58]:
#Feed the prepared test data into the loaded model
rf_model_loaded.predict(X_test)

array(['Breaded Pork Tenderloin Sandwich',
       'Ultimate Grilled Cheese Sandwich (with bacon and tomato)',
       'Fried Catfish Basket', 'Indiana Pork Chili',
       'Hoosier BBQ Pulled Pork Sandwich', 'Sweet Potato Fries',
       'Cornbread Hush Puppies', 'Hoosier BBQ Pulled Pork Sandwich',
       'Fried Catfish Basket', 'Indiana Pork Chili', 'Sweet Potato Fries',
       'Fried Catfish Basket', 'Cornbread Hush Puppies',
       'Sugar Cream Pie', 'Sweet Potato Fries', 'Sugar Cream Pie',
       'Breaded Pork Tenderloin Sandwich',
       'Indiana Corn on the Cob (brushed with garlic butter)',
       'Hoosier BBQ Pulled Pork Sandwich', 'Indiana Pork Chili',
       'Sugar Cream Pie', 'Hoosier BBQ Pulled Pork Sandwich',
       'Indiana Pork Chili', 'Sweet Potato Fries',
       'Cornbread Hush Puppies', 'Indiana Pork Chili',
       'Sweet Potato Fries', 'Hoosier BBQ Pulled Pork Sandwich',
       'Sugar Cream Pie', 'Indiana Pork Chili', 'Fried Catfish Basket',
       'Cornbread Hush Puppi